In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo



conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)



In [2]:
# Define database and collection
db = client.mars_db
collection = db.pages

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
articles = soup.find_all('div', class_='slide')

for article in articles:
    title = article.find('div', class_='content_title').text
    paragraph = article.find('div', class_= 'rollover_description_inner').text
    
    print('---------------')
    print(title)
    print(paragraph)
    
    
    post = {
        'title': title,
        'paragraph': paragraph,
        }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)


In [17]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

browser.click_link_by_id('full_image')

